In [22]:
import sys
sys.path.append("..")
import torch
import torch.nn as nn
from Tensorized_components.patch_embedding  import Patch_Embedding     
from Tensorized_components.w_msa  import WindowMSA     
from Tensorized_components.sh_wmsa  import ShiftedWindowMSA     
from Tensorized_components.patch_merging  import TensorizedPatchMerging  
from Tensorized_Layers.TCL_CHANGED import TCL_CHANGED   


In [23]:
# TODO: add device compatible

In [ ]:
import sys
sys.path.append("..")
import torch
import torch.nn as nn
from Tensorized_components.patch_embedding  import Patch_Embedding     
from Tensorized_components.w_msa  import WindowMSA     
from Tensorized_components.sh_wmsa  import ShiftedWindowMSA     
from Tensorized_components.patch_merging  import TensorizedPatchMerging  
from Tensorized_Layers.TCL_CHANGED import TCL_CHANGED   


class SwinTransformer(nn.Module):
    def __init__(self,
                 img_size=224,
                 patch_size=4,
                 in_chans=3,
                 embed_shape=(4,4,3),
                 bias=True,
                 dropout=0.5,
                 device="cpu"):
        super(SwinTransformer, self).__init__()

        self.norm1 = nn.LayerNorm(embed_shape)
        self.dropout = nn.Dropout(dropout)


        self.patch_embedding = Patch_Embedding(img_size=img_size,
                                                patch_size=patch_size,
                                                in_chans=in_chans,
                                                embed_shape=embed_shape,
                                                bias=bias)
    
        

        self.w_msa_1 = WindowMSA(window_size=7, embed_dims=(4, 4, 3),
                    rank_window=(4,4,3), head_factors=(2,2,1), device=device).to(device)
        
        self.sw_msa_1 = ShiftedWindowMSA(
            window_size=7,embed_dims=(4, 4, 3),rank_window=(4, 4, 3),head_factors=(2, 2, 1),device='cpu')
    
        self.patch_merging_1 = TensorizedPatchMerging(
        input_size=(16,56, 56, 4, 4, 3),
        in_embed_shape=embed_shape,
        out_embed_shape=(4,4,6),   
        bias=bias,
        ignore_modes=(0,1,2),
        device=device).to(device)

        self.tcl_1 = TCL_CHANGED(
            input_size=(1, 56, 56, 4, 4, 3),  # Example shape after ignoring batch dim
            rank=(4, 4, 3),
            ignore_modes=(0, 1, 2),
            bias=True,
            device='cpu'
        )

        
    def forward(self, x):

        # stage 1
        patches = self.patch_embedding(x)
        print("data size after patch",patches.shape)

        # wmsa_1

        x_res = patches
        x = self.norm1(patches)
        x = self.dropout(self.w_msa_1(x))
        x = x + x_res


        x_res = x
        x = self.norm1(x)
        x = self.tcl_1(x)
        x = x + x_res

        x_res = x
        x = self.norm1(x)
        x = self.dropout(self.sw_msa_1(x))
        x = x + x_res

        x_res = x
        x = self.norm1(x)
        x = self.tcl_1(x)
        x = x + x_res

        print("shape of x is",  x.shape)


  







        # x_res = patches

        # print("shape of patches is ",patches.shape)

        # x = self.norm1(patches + self.dropout(self.w_msa_1(patches)))

        # print("shape after window attention",x.shape)
        # wmsa_1 = x + x_res

        # sh_wmsa_1

        # print("output wmsa stage 1" , wmsa_1.shape)


        # end stage 1

        # patch_merginig_1 = self.patch_merging_1(wmsa_1)

        # print("patch merging",patch_merginig_1.shape)


        return patches

In [25]:
import torch

# Create a dummy input tensor (batch_size=1, channels=3, height=224, width=224)
dummy_input = torch.randn(1, 3, 224, 224)

# Initialize the model
model = SwinTransformer(img_size=224,patch_size=4,in_chans=3,embed_shape=(4,4,3),bias=True,device="cpu")

# Forward pass
output = model(dummy_input)

# Output shape
# print(output.shape)

shift_size is 3
data size after patch torch.Size([1, 56, 56, 4, 4, 3])
shape of attention flat is torch.Size([1, 8, 8, 2, 2, 1, 49, 49])
shape of bias is torch.Size([1, 1, 1, 2, 2, 1, 49, 49])
attention flat after add bias torch.Size([1, 8, 8, 2, 2, 1, 49, 49])
shape of mask is torch.Size([1, 8, 8, 1, 1, 1, 49, 49])
attn_flat shape afeter add to the mask torch.Size([1, 8, 8, 2, 2, 1, 49, 49])
shift size in reverse 3
shape of x is torch.Size([1, 56, 56, 4, 4, 3])


In [26]:
import torch
import sys
sys.path.append("..")
from Tensorized_components.Shifted_window_partition  import ShiftedWindowPartition     
from Tensorized_components.sh_wmsa  import ShiftedWindowMSA


x = torch.arange(64).reshape(1, 8, 8, 1, 1, 1)

# Create a sample tensor with shape (B, H, W, C)
# B, H, W, C = 1, 8, 8, 3  # Ensure H and W are divisible by the window_size.
# x = torch.arange(B * H * W * C, dtype=torch.float32).reshape(B, H, W, C)
print("Original tensor shape:", x.shape)

# Set the window size and shift size
window_size = 4
shift_size = 2

# Instantiate the ShiftedWindowPartition module.
shifted_window_partition = ShiftedWindowPartition(window_size, shift_size)

# Partition the tensor with a spatial shift.
windows = shifted_window_partition(x)
print("Partitioned windows shape:", windows.shape)
print("Partitioned windows shape:", windows[0,0,0,:,:,0,0,0])

# Reverse the window partition operation to reconstruct the original tensor.
# x_reconstructed = shifted_window_partition.reverse(windows, H, W)
# print("Reconstructed tensor shape:", x_reconstructed.shape)

    # Verify that the reconstruction matches the original tensor.
    # if torch.allclose(x, x_reconstructed):
        # print("Success: The reconstructed tensor matches the original tensor.")
    # else:
        # print("Error: The reconstructed tensor does not match the original tensor.")

Original tensor shape: torch.Size([1, 8, 8, 1, 1, 1])
Partitioned windows shape: torch.Size([1, 2, 2, 4, 4, 1, 1, 1])
Partitioned windows shape: tensor([[18, 19, 20, 21],
        [26, 27, 28, 29],
        [34, 35, 36, 37],
        [42, 43, 44, 45]])


In [27]:
# usage
device = "cpu"
x = torch.randn(1, 56, 56, 4, 4, 3, device=device)

# Window parameters.
window_size = 7
# TCL configuration: for each window, input size is (7, 7, 4, 4, 3).
# Here, we set rank_window to (4, 4, 3) and head_factors to (2, 2, 1) so that
# the number of heads is 2*2*1 = 4.
rank_window = (4, 4, 3)
head_factors = (2, 2, 1)

# Instantiate the W-MSA module.
w_msa = ShiftedWindowMSA(window_size=window_size, embed_dims=(4, 4, 3),
        rank_window=rank_window, head_factors=head_factors, device=device).to(device)

x = w_msa(x)




shift_size is 3
shape of attention flat is torch.Size([1, 8, 8, 2, 2, 1, 49, 49])
shape of bias is torch.Size([1, 1, 1, 2, 2, 1, 49, 49])
attention flat after add bias torch.Size([1, 8, 8, 2, 2, 1, 49, 49])
shape of mask is torch.Size([1, 8, 8, 1, 1, 1, 49, 49])
attn_flat shape afeter add to the mask torch.Size([1, 8, 8, 2, 2, 1, 49, 49])
shift size in reverse 3
